# Load Test deployed web application

In this notebook, we test the latency of the deployed web application by sending a number of duplicate questions as asychronous requests.

In [2]:
import warnings
warnings.filterwarnings(action="ignore", message="numpy.dtype size changed")

import asyncio
import json
import random
import urllib.request
from timeit import default_timer

import aiohttp
from tqdm import tqdm
import requests
import pandas as pd
from utilities import text_to_json

In [3]:
print(aiohttp.__version__) 

3.3.2


We will test our deployed service with 100 calls. We will only have 4 requests concurrently at any time. We have only deployed one pod on one node and increasing the number of concurrent calls does not really increase throughput. Feel free to try different values and see how the service responds.

In [4]:
NUMBER_OF_REQUESTS = 100  # Total number of requests
CONCURRENT_REQUESTS = 4   # Number of requests at a time

Get the IP address of our service.

In [5]:
service_json = !kubectl get service azure-ml -o json
service_dict = json.loads(''.join(service_json))
app_url = service_dict['status']['loadBalancer']['ingress'][0]['ip']

In [6]:
scoring_url = 'http://{}/score'.format(app_url)
version_url = 'http://{}/version'.format(app_url)
health_url = 'http://{}/'.format(app_url)

In [7]:
!curl $health_url

Healthy

In [8]:
!curl $version_url # Reports the lightgbm version

2.1.2

In [9]:
dupes_test_path = 'dupes_test.tsv'
dupes_test = pd.read_csv(dupes_test_path, sep='\t', encoding='latin1')
dupes_to_score = dupes_test.iloc[:NUMBER_OF_REQUESTS,4]

In [10]:
url_list = [[scoring_url, jsontext] for jsontext in dupes_to_score.apply(text_to_json)]

In [11]:
def decode(result):
    return json.loads(result.decode("utf-8"))

In [12]:
async def fetch(url, session, data, headers):
    start_time = default_timer()
    async with session.request('post', url, data=data, headers=headers) as response:
        resp = await response.read()
        elapsed = default_timer() - start_time
        return resp, elapsed

In [13]:
async def bound_fetch(sem, url, session, data, headers):
    # Getter function with semaphore.
    async with sem:
        return await fetch(url, session, data, headers)

In [14]:
async def await_with_progress(coros):
    results=[]
    for f in tqdm(asyncio.as_completed(coros), total=len(coros)):
        result = await f
        results.append((decode(result[0]),result[1]))
    return results

In [15]:
async def run(url_list, num_concurrent=CONCURRENT_REQUESTS):
    headers = {'content-type': 'application/json'}
    tasks = []
    # create instance of Semaphore
    sem = asyncio.Semaphore(num_concurrent)

    # Create client session that will ensure we dont open new connection
    # per each request.
    async with aiohttp.ClientSession() as session:
        for url, data in url_list:
            # pass Semaphore and session to every POST request
            task = asyncio.ensure_future(bound_fetch(sem, url, session, data, headers))
            tasks.append(task)
        return await await_with_progress(tasks)

Below we run the 100 requests against our deployed service.

In [16]:
loop = asyncio.get_event_loop()
start_time = default_timer()
complete_responses = loop.run_until_complete(asyncio.ensure_future(run(url_list, num_concurrent=CONCURRENT_REQUESTS)))
elapsed = default_timer() - start_time
print('Total Elapsed {}'.format(elapsed))
print('Avg time taken {0:4.2f} ms'.format(1000*elapsed/len(url_list)))

100%|██████████| 100/100 [00:08<00:00, 11.57it/s]

Total Elapsed 8.650007339194417
Avg time taken 86.50 ms


In [17]:
# Example response
complete_responses[0]

({'result': [[[27928, 27943, 0.9994291873318922],
    [14220321, 14220323, 0.3777013664717283],
    [3384504, 3384534, 0.0008692544514442122],
    [20279484, 20279485, 0.00018390175149079251],
    [824349, 3354511, 0.00014506188508628983],
    [23667086, 23667087, 0.00013645228359681323],
    [336859, 336868, 0.000118945270994279],
    [1451009, 1451043, 8.258517728346488e-05],
    [1051782, 1051797, 8.250338021331798e-05],
    [403967, 22745553, 6.153863170894845e-05],
    [11922383, 11922384, 5.1157447215651005e-05],
    [750486, 750506, 5.059878338976614e-05],
    [111102, 111111, 3.258347291447318e-05],
    [205853, 553734, 3.091903814356007e-05],
    [4968406, 4968448, 2.8814241790668893e-05],
    [13840429, 13840431, 2.8794792285915736e-05],
    [364952, 364997, 1.8629629530953053e-05],
    [3034941, 3034952, 1.752737388881987e-05],
    [5874652, 5876747, 1.7186203770500705e-05],
    [1873983, 1873999, 1.6984465096944128e-05],
    [152975, 153047, 1.2085550446551274e-05],
    [74

Let's use the number of original questions to count the succesful responses.

In [18]:
no_questions = len(complete_responses[0][0]['result'][0])

In [19]:
num_succesful=[len(i[0]['result'][0]) for i in complete_responses].count(no_questions)
print('Succesful {} out of {}'.format(num_succesful, len(url_list)))

Succesful 100 out of 100


Next, we will explore the real-time scoring in an [iPyWidget app](08_Real_Time_Scoring.ipynb).